In [2]:

import vaex 

In [3]:
calendar=vaex.open("../data/raw/calendar.csv")
train=vaex.open("../data/raw/sales_train_evaluation.csv")
prices=vaex.open("../data/raw/sell_prices.csv")


<frozen importlib._bootstrap>:228: RuntimeWarning:

numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 96 from PyObject



In [7]:
import pandas as pd
import dtale
d = dtale.show(calendar.to_pandas_df())
d.open_browser()

In [ ]:
train

In [ ]:
prices

In [ ]:
prices_calendar = prices.join(calendar,  on=['wm_yr_wk', 'd'], lprefix='p_', rprefix='c_', allow_duplication=True)
prices_calendar